In [1]:
import cv2
import numpy as np
import torch
import pandas as pd
from datetime import datetime
from ultralytics import YOLO

# Load YOLOv8 model (use GPU if available)
model = YOLO('yolov8n.pt').to('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize video capture
cap = cv2.VideoCapture('C:\\Users\\donka\\Downloads\\cars.mp4')

# Get original video dimensions and frame rate
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer for saving the output
out = cv2.VideoWriter('vcoutput.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Create or load the Excel file
excel_file = 'vehicle_data.xlsx'
columns = ['Vehicle_ID', 'Time', 'Date', 'Speed_km/h']

# Initialize a DataFrame to store vehicle details
vehicle_data = pd.DataFrame(columns=columns)

# Counting variables
vehicle_count = 0
line_position = frame_height * 2 // 3  # Adjusted line position

# Vehicle classes according to COCO dataset (car, motorcycle, bus, truck)
vehicle_classes = [2, 3, 5, 7]

# Tracker to track vehicle centroids
vehicle_tracker = {}

# Vehicle IDs for counting and tracking
next_vehicle_id = 0

def is_vehicle(class_id):
    return int(class_id) in vehicle_classes

def calculate_speed(distance_pixels, fps):
    # Assume an approximate conversion factor for pixel distance to km/h
    pixel_to_kmh = 0.1  # Placeholder factor (this needs calibration based on your video and camera setup)
    speed = (distance_pixels * fps * pixel_to_kmh)
    return speed

def process_detections(results, frame, frame_number):
    global vehicle_count, next_vehicle_id, vehicle_tracker, vehicle_data
    detections = results[0].boxes.data.cpu().numpy()  # Convert to NumPy for easier processing
    new_centroids = []

    for detection in detections:
        if len(detection) >= 6:
            x1, y1, x2, y2, confidence, class_id = detection[:6]
            if is_vehicle(class_id):
                # Draw bounding box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

                # Display the confidence score above the bounding box with larger size and red color
                cv2.putText(frame, f'{confidence:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

                # Calculate centroid of the bounding box
                x_center = int((x1 + x2) / 2)
                y_center = int((y1 + y2) / 2)
                new_centroids.append((x_center, y_center))

    # Match current detections with existing trackers
    updated_tracker = {}
    for vehicle_id, (prev_centroid, counted, initial_frame) in vehicle_tracker.items():
        distances = [np.linalg.norm(np.array(prev_centroid) - np.array(new_c)) for new_c in new_centroids]
        if distances:
            min_dist_idx = np.argmin(distances)
            min_dist = distances[min_dist_idx]

            # If the closest match is within a reasonable distance, update tracker
            if min_dist < 50:  # Threshold distance
                new_centroid = new_centroids.pop(min_dist_idx)
                updated_tracker[vehicle_id] = (new_centroid, counted, initial_frame)
                # Check if the vehicle has crossed the line
                if not counted and prev_centroid[1] < line_position <= new_centroid[1]:
                    vehicle_count += 1
                    updated_tracker[vehicle_id] = (new_centroid, True, initial_frame)

                    # Store vehicle information in Excel
                    time = datetime.now().strftime('%H:%M:%S')
                    date = datetime.now().strftime('%Y-%m-%d')
                    distance_traveled = np.linalg.norm(np.array(new_centroid) - np.array(prev_centroid))
                    time_elapsed = (frame_number - initial_frame) / fps
                    speed_kmh = calculate_speed(distance_traveled / time_elapsed, fps)
                    vehicle_data = vehicle_data.append({
                        'Vehicle_ID': f'car_id{vehicle_id:03d}',
                        'Time': time,
                        'Date': date,
                        'Speed_km/h': round(speed_kmh, 2)
                    }, ignore_index=True)

    # Add new detections as new vehicles if no match was found
    for new_centroid in new_centroids:
        updated_tracker[next_vehicle_id] = (new_centroid, False, frame_number)
        next_vehicle_id += 1

    vehicle_tracker = updated_tracker
    return frame

# Ensure resources are cleaned up if error occurs
try:
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Draw the counting line
        cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 0, 255), 2)  # Red line

        # Process detections and count vehicles
        frame = process_detections(results, frame, frame_count)

        # Display the vehicle count at the top of the screen in red
        cv2.putText(frame, f'Vehicle Count: {vehicle_count}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show the frame and write it to output video
        cv2.imshow('Vehicle Detection and Counting', frame)
        out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save the vehicle data to an Excel file
    vehicle_data.to_excel(excel_file, index=False)

print(f"Total vehicle count: {vehicle_count}")
print(f"Vehicle data saved to {excel_file}")



0: 384x640 7 cars, 1 bus, 3 trucks, 497.5ms
Speed: 24.4ms preprocess, 497.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 buss, 1 truck, 442.8ms
Speed: 8.6ms preprocess, 442.8ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 truck, 448.3ms
Speed: 79.8ms preprocess, 448.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 772.1ms
Speed: 10.1ms preprocess, 772.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 trucks, 491.1ms
Speed: 8.0ms preprocess, 491.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 544.0ms
Speed: 10.2ms preprocess, 544.0ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 bus, 1 truck, 954.0ms
Speed: 0.0ms preprocess, 954.0ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)


AttributeError: 'DataFrame' object has no attribute 'append'

In [15]:
pip install openpyxl


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
